In [22]:
import os
import PyPDF2
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.enums import TA_JUSTIFY

In [23]:
# Import groq API key from .env file
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [24]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [25]:
def create_qa_system(past_paper_paths, subject_book_paths):
    all_texts = ""
    for pdf_path in past_paper_paths + subject_book_paths:
        pdf_text = extract_text_from_pdf(pdf_path)
        all_texts += pdf_text + "\n"

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(all_texts)

    embeddings = HuggingFaceEmbeddings()
    # see dimensions of the embeddings and texts
    print(embeddings.embed(texts[0]).shape)
    print(texts[0])
    db = Chroma.from_texts(texts, embeddings)

    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0.7,
    )

    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever(search_kwargs={"k": 3}))

    return qa

In [26]:
# def generate_question_paper(qa_system, num_questions):
#     questions = []
#     for i in range(num_questions):
#         prompt = f"Generate a new subjective question paper based on the contents of the past papers, ensuring it's within the scope of the subject books. There should be two sections in the paper, in first section there are three parts of short questions and in each part of short question, there are six short questions. Then in second section, there are three long questions. The question should be challenging but fair. Format the question in a structured manner suitable for an exam paper. also space between each question should be at least 2 lines."
#         response = qa_system.run(prompt)
#         questions.append(response)
#     return questions

In [27]:
def generate_question_paper(qa_system, num_questions):
    questions = []
    for i in range(num_questions):
        prompt = f"Generate a new subjective question paper based on the contents of the past papers, ensuring it's within the scope of the subject books. There should be two sections in the paper, in first section there are three parts of short questions and in each part of short question, there are six short questions. Then in second section, there are three long questions. The question should be challenging but fair. Format the question in a structured manner suitable for an exam paper. also space between each question should be at least 2 lines. donot repeat the questions."
        response = qa_system.run(prompt)
        questions.append(response)
    return questions

In [28]:
def create_reformatted_question_paper_pdf(questions, output_path):
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    content = []

    content.append(Paragraph("Generated Question Paper", styles['Title']))
    content.append(Spacer(1, 12))

    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    
    for i, question in enumerate(questions, 1):
        content.append(Paragraph(f"Question {i}", styles['Heading2']))
        lines = question.split('\n')
        for line in lines:
            if line.strip():
                if line.startswith('*') or line.startswith('-'):
                    content.append(Paragraph(f"• {line.strip('*- ')}", styles['BodyText']))
                else:
                    content.append(Paragraph(line, styles['Justify']))
        content.append(Spacer(1, 12))

    doc.build(content)

In [29]:
import os

def get_pdf_files_from_folder(folder_path):
    """
    Returns a list of PDF file paths from the specified folder.
    """
    pdf_files = []
    for file in os.listdir(folder_path):
        if file.lower().endswith('.pdf'):
            pdf_files.append(os.path.join(folder_path, file))
    return pdf_files

In [30]:
def main():
    past_paper_folder = "sub_output_pdfs"  # Folder containing past paper PDFs
    past_paper_paths = get_pdf_files_from_folder(past_paper_folder)
    
    subject_book_paths = ["cs9.pdf"]  # Add your subject book PDFs here
    
    qa_system = create_qa_system(past_paper_paths, subject_book_paths)
    
    num_questions = 1  # You can adjust this number
    questions = generate_question_paper(qa_system, num_questions)
    
    output_pdf_path = "new_subjective question paper.pdf"
    create_reformatted_question_paper_pdf(questions, output_pdf_path)
    
    print(f"Generated question paper has been saved to {output_pdf_path}")
    print(f"Used {len(past_paper_paths)} past papers from the folder '{past_paper_folder}'")

if __name__ == "__main__":
    main()

Created a chunk of size 25338, which is longer than the specified 1000
C:\Users\zubay\AppData\Local\Temp\ipykernel_16204\2691119666.py:10: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\zubay\.conda\envs\fyp\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'embed'

In [30]:
# def main():
#     past_paper_paths = ["output.pdf", "output1.pdf"]  # Your past paper PDFs
#     subject_book_paths = ["cs10.pdf"]  # Add your subject book PDFs here
    
#     qa_system = create_qa_system(past_paper_paths, subject_book_paths)
    
#     num_questions = 10  # You can adjust this number
#     questions = generate_question_paper(qa_system, num_questions)

#     output_pdf_path = "r1_generated_question_paper.pdf"
#     create_reformatted_question_paper_pdf(questions, output_pdf_path)
#     print(f"Generated question paper has been saved to {output_pdf_path}")

# if __name__ == "__main__":
#     main()